In [ ]:
import tensorflow as tf
import numpy as np
import sys

In [ ]:
input_tensor = tf.keras.Input(shape=(512,640,9))
x = input_tensor
def down(filters, apply_batchnorm=True):    
    initializer = tf.random_normal_initializer(0.,0.02)
    a = tf.keras.models.Sequential()
    a.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=(4,4), padding='same', strides=1, ))
    a.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
    a.add(tf.keras.layers.BatchNormalization())
    a.add(tf.keras.layers.LeakyReLU())
    return a

In [ ]:
down_stack = []

In [ ]:
for i in range(4):
    b = down(64*(2**i))
    down_stack.append(b)

for i in range(3):
    b = down(512)
    down_stack.append(b)

In [ ]:
last = x
skips = []
for c in down_stack:
    last = c(last)
    skips.append(last)

In [ ]:
skips = reversed(skips[:-1])

In [ ]:
def up(filters):
    initializer = tf.random_normal_initializer(0.,.02)
    tensor = tf.keras.models.Sequential()
    tensor.add(tf.keras.layers.Conv2DTranspose(filters=filters, kernel_size=4, strides=2, padding='same', kernel_initializer=initializer))
    tensor.add(tf.keras.layers.BatchNormalization())
    tensor.add(tf.keras.layers.ReLU())
    return tensor

In [ ]:
up_stack = []

In [ ]:
for i in range(3):
    up_stack.append(up(512))

for i in range(3):
    up_stack.append(up(512/(2**(i+1))))


In [ ]:
y = last
for u, skip in zip(up_stack, skips):
    y = u(y)
    y = tf.keras.layers.Concatenate()([y, skip])

In [ ]:
z = up(32)
y = z(y)
print(y.shape)
y = tf.keras.layers.Concatenate()([x, y])
y = tf.keras.layers.Conv2D(filters=3, kernel_size=4, strides=1, padding='same', activation='tanh')(y)

In [ ]:
model = tf.keras.Model(inputs=input_tensor, outputs=y)

In [ ]:
model = tf.keras.Model(inputs=input_tensor, outputs=y)
model.summary()
tf.keras.util.plot_model(model, show_shape=True, dpi=64)

In [ ]:
import cv2 as cv
def preprocess(image):
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    image = (image/127.5) -1
    retval = tf.expand_dims(image, axis=0)
    return retval

def concat_all_3(images):
  return tf.concat(images, 3)

def pad2(image): # get the image subtraction to a multiple of (640, 512)
  border_h_0 = int((3072-2825)/2)
  border_h_1 = border_h_0 + 1
  width_crop_half = 17, 18
  shape = image.shape
  destination = np.zeros([3072, 3840, 3], dtype=np.uint8)
  destination[123:3072-124, ::, ::] += image[::, 17:-18:,::]
  return cv.resize(destination, (640,512))
  

In [ ]:
import matplotlib.pyplot as plt
path = "/home/computer/CS/DroneProject/im2im_labeling/images/"

In [ ]:
# only run the following code when you need to preprocess the images. this might take a while
count = 30000

for i in range(3993, count):
  noir = pad2(cv.imread(path + "unfiltered/NoIR_" + str(i) + ".jpg"))
  optical = pad2(cv.imread(path + "filtered/Optical_" + str(i) + ".jpg"))
  subtraction = pad2(cv.imread(path + "subtractions/subtraction_" + str(i) + ".jpg"))
  #thermal = cv.imread(path + "thermal/thermal_" + str(i) + ".jpg")
  cv.imwrite(path+"unfiltered_preprocessed/NoIR_" + str(i) + ".jpg", noir)
  cv.imwrite(path+"filtered_preprocessed/Optical_" + str(i) + ".jpg", optical)
  cv.imwrite(path+"subtractions_preprocessed/subtraction_" + str(i) + ".jpg", subtraction)

In [ ]:
# define the discriminator network
discrim_input0 = tf.keras.layers.Input(shape=(512, 640, 3))
discrim_input1 = tf.keras.layers.Input(shape=(512,640, 9))
conc = tf.keras.layers.Concatenate()([discrim_input0, discrim_input1])
x = conc
initializer = tf.random_normal_initializer(0.0, .02)
for i in range(4):
    x = tf.keras.layers.Conv2D(filters=256*pow(2, -(i+1)), kernel_size=(3,3), padding='same', strides=1)(x)
    x = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2))(x)
    x = tf.keras.layers.LeakyReLU()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Conv2D(filters=1, kernel_size=3, strides=1, padding='same', kernel_initializer=initializer)(x)

In [ ]:
discrim = tf.keras.Model(inputs=[discrim_input0, discrim_input1], outputs=x)
discrim.summary()
tf.keras.utils.plot_model(discrim, show_shapes=True)

In [ ]:
import random
num_epochs= 30
batch_size = 1 #construct a cv mat with given size python
xentropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
mse = tf.keras.losses.MeanSquaredError()
l1 = lambda y_true, y_pred : tf.reduce_mean(tf.abs(y_true - y_pred))
gen_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discrim_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=.5)

In [ ]:
batch_size = 1

In [ ]:
def paths_for(i):
    lst = []
    lst.append(path+"unfiltered_preprocessed/NoIR_" + str(i) + ".jpg")
    lst.append(path+"filtered_preprocessed/Optical_" + str(i) + ".jpg")
    lst.append(path+"subtractions_preprocessed/subtraction_" + str(i) + ".jpg")
    lst.append(path + "thermal/thermal_" + str(i) + ".jpg")
    return lst

In [ ]:
valid_indices = [i for i in range(0,3924)]
for i in range(3993, 41772):
    valid_indices.append(i)
    
for i in range(41978, 120000):
    valid_indices.append(i)

In [ ]:
random.shuffle(valid_indices)

In [ ]:
def preprocess_example(i):
    paths_for_i = paths_for(i)
    all_four = [preprocess(cv.imread(path)) for path in paths_for_i]
    x = concat_all_3(all_four[:-1])
    return x, all_four[-1]

In [ ]:
def image_generator(image_paths): 
    for example in image_paths:
        all_four = [preprocess(cv.imread(path2)) for path2 in example]
        input_tensr = concat_all_3(all_four[:-1])
        output = all_four[-1]
        yield np.array(input_tensr), np.array(output)

In [ ]:
image_paths = [] # for the train data. hold out the other 20%
for idx in valid_indices[:96000]:
    image_paths.append(paths_for(idx))

In [ ]:
# define the data set
dataset = tf.data.Dataset.from_generator(lambda: image_generator(image_paths), output_types=(tf.float32, tf.float32), output_shapes=([None, 512, 640, 9], [None, 512, 640, 3]))
#dataset = dataset.map(preprocess_example)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
import time
def post_scale(im):
    im = ((im*127.5) + 127.5).numpy()
    return im

In [ ]:
# WARNING: this code block loads weights into the model and the current weights are not guarenteed to be saved

latest = 'epoch_6_12'
model.load_weights(latest)

In [ ]:
l = 200
l_w = 50
l_b = l_w
for epoch_i in range(0,num_epochs):
    dataset.shuffle(buffer_size=10)
    t0 = time.time()
    epoch_gen_loss = 0.0
    epoch_discrim_loss = 0.0
    epoch_mse_loss = 0.0
    batch_num = 0
    with tf.device("/GPU:0"):
        for batch_x, batch_y in dataset:
            if batch_num % 1000 == 0:
                print("Batch number=", batch_num)
            with tf.GradientTape() as discrim_tape, tf.GradientTape() as gen_tape:
                batch_gen_loss = 0.0
                batch_discrim_loss = 0.0
                for x, y in zip(batch_x, batch_y):
                    #print(x.shape)
                    y_pred = model(x, training=True)
                    l2 = mse(y, y_pred)
                    epoch_mse_loss += l2
                    discrim_output_fake = discrim([y_pred, x], training=True)
                    output_asif_real = tf.ones_like(discrim_output_fake)
                    batch_gen_loss += l*l2 + xentropy(output_asif_real, discrim_output_fake)
                    discrim_output_real = discrim([y, x], training=True)
                    fake_logits = tf.zeros_like(discrim_output_real)
                    batch_discrim_loss += xentropy(output_asif_real, discrim_output_real) + xentropy(fake_logits, discrim_output_fake)
                
                gen_grad = gen_tape.gradient(batch_gen_loss, model.trainable_variables)
                gen_optimizer.apply_gradients(grads_and_vars=zip(gen_grad, model.trainable_variables))
                
                discrim_grad = discrim_tape.gradient(batch_discrim_loss, discrim.trainable_variables)
                discrim_optimizer.apply_gradients(grads_and_vars=zip(discrim_grad, discrim.trainable_variables))
                
                epoch_gen_loss += batch_gen_loss
                epoch_discrim_loss += batch_discrim_loss
                
                batch_num += 1
            
    print('Epoch ', epoch_i, 'Generator Loss: ', epoch_gen_loss.numpy()/(2*len(image_paths)), ' Discriminator Loss: ', epoch_discrim_loss.numpy()/(2*len(image_paths)), 'MSE loss: ', epoch_mse_loss.numpy()/(2*len(image_paths)))
    print("Time taken: ", time.time()-t0)
    model.save_weights('./epoch_'+str(epoch_i) + "_13")

In [ ]:
def test_paths_for(i):
    lst = []
    lst.append(path+"unfiltered_preprocessed/NoIR_" + str(i) + ".jpg")
    lst.append(path+"filtered_preprocessed/Optical_" + str(i) + ".jpg")
    lst.append(path+"subtractions_preprocessed/subtraction_" + str(i) + ".jpg")
    lst.append(path + "thermal/thermal_" + str(i) + ".jpg")
    return lst

In [ ]:
test_valid_indices = valid_indices[96000:]

In [ ]:
test_image_paths = []
for idx in test_valid_indices:
    test_image_paths.append(test_paths_for(idx))

In [ ]:
test_dataset = tf.data.Dataset.from_generator(lambda: image_generator(test_image_paths), output_types=(tf.float32, tf.float32), output_shapes=([None, 512, 640, 9], [None, 512, 640, 3]))
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def percent_difference(predicted, thermal):
    predicted = predicted[0].numpy()
    thermal = thermal[0].numpy()
    abs_dif = cv.absdiff(predicted, thermal)
    dif_r = abs_dif[::,::,2]
    dif_g = abs_dif[::,::,1]
    dif_b = abs_dif[::,::,0]
    percent_dif_red = np.divide(dif_r, thermal[::,::, 2], out=np.zeros_like(dif_r), where=thermal[::,::,2]!=0)
    percent_dif_red =  np.mean(percent_dif_red)
    
    percent_dif_green = np.divide(dif_g, thermal[::,::,1], out=np.zeros_like(dif_g), where=thermal[::,::,2]!=0)
    percent_dif_green = np.mean(percent_dif_green)
    
    percent_dif_blue = np.divide(dif_b, thermal[::,::,0], out=np.zeros_like(dif_b), where=thermal[::,::,2]!=0)
    percent_dif_blue = np.mean(percent_dif_blue)
    
    return [percent_dif_red, percent_dif_green, percent_dif_blue]

In [ ]:
# run this code to test

percent_diffs = [0.0, 0.0, 0.0]
n = 0
good = False
for batch_x, batch_y in test_dataset:
    if good:
        break
    for x,y in zip(batch_x, batch_y):
        y_pred = model(x)
        difs = percent_difference(y_pred, y)
        percent_diffs[0] += difs[0]
        percent_diffs[1] += difs[1]
        percent_diffs[2] += difs[2]
        n+=1
        if n >= 5000:
            good = True
            
        
print("Average percent difference: ", [abs(percent_diff/n) for percent_diff in percent_diffs])